In [1]:
import numpy as np
import os
import math
import cv2
np.random.seed(20)

Find Avg, Max, Var of Acceleration/Yaw Rate and Num Frames

In [6]:
target = os.listdir('../Datasets/hdd_data/target')
target.remove('README.md')
target.remove('.README.md.swp')
target.sort()
sensors = os.listdir('../Datasets/hdd_data/sensor')
sensors.remove('README.md')
sensors.remove('.README.md.swp')
sensors.sort()
imgs = os.listdir('../Datasets/hdd_data/camera')
imgs.remove('.DS_Store')
imgs.sort()

def avg_num_frames(action: int):
    for j in target:
        frame_count = 0
        count = 0
        arr = np.load('../Datasets/hdd_data/target/%s' % j, allow_pickle=True)
        for i in range(round(len(arr) * 0.8)):
            if arr[i] == action:
                frame_count += 1
            if arr[i] == action and arr[i-1] != action:
                count += 1
        if count != 0:
            print(round(frame_count/count, 3))
        else:
            print('nan')

# mode: 0 - max, 1 - avg, 2 - var
# sensor: 0 for acceleration, 7 for yaw rate
def calc_sensor_data(action: int, stat: int, sensor: int):
    assert stat in [0, 1, 2]
    assert sensor >= 0 and sensor <= 7
    for j in target:
        target_arr = np.load('../Datasets/hdd_data/target/%s' % j, allow_pickle=True)
        sensor_arr = np.load('../Datasets/hdd_data/sensor/%s' % j, allow_pickle=True)
        sensor_acc = []
        for i in range(round(len(target_arr) * 0.8)):
            if target_arr[i] == action:
                sensor_acc.append(sensor_arr[i][sensor])
        if len(sensor_acc):
            if stat == 0:
                print(np.max(sensor_acc))
            elif stat == 1:
                print(np.mean(sensor_acc))
            else:
                print(np.var(sensor_acc))
        else:
            print('nan')
    

In [7]:
avg_num_frames(10)

0
0
0
9.0
9.0
6.0
8.0
6.0
6.0
0
0
6.0
8.0
12.0
0
8.0
0
0
0
0
0
0
0
9.0
0
0
6.0
4.0
3.0
4.5
6.0
6.0
12.0
3.0
0
0
13.0
0
0
9.5
9.0
9.0
9.0
15.0
0
0
0
0
0
16.5
0
15.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6.0
0
0
0
0
6.0
13.5
9.0
13.0
18.0
0
13.0
14.0
10.5
9.0
0
0
0
0
0
0
0
0
0
10.0
15.0
25.5
0
6.0
5.25
0
6.0
0
12.0
0
0
0
0
7.0
0
9.0
3.0
0
6.0
21.0
7.5
0
3.0
0
13.5
4.0
9.0
0
0
9.0
0
0
0
0
3.0
13.5
13.5
3.0
15.0
9.0
18.0
3.0
0
0
45.0


In [231]:
yolo = cv2.dnn.readNetFromDarknet('../FYP/darknet/cfg/yolov3.cfg', '../FYP/darknet/yolov3.weights')

In [ ]:
for j in target[:1]:
        target_arr = np.load('../Datasets/hdd_data/target/%s' % j, allow_pickle=True)
        sensor_arr = np.load('../Datasets/hdd_data/sensor/%s' % j, allow_pickle=True)
        sensor_acc = []
        for i in range(round(len(target_arr) * 0.8)):
            sensor_acc.append(sensor_arr[i][3])
print(sensor_acc)

In [4]:
img = cv2.imread('hdd_data/camera/201702271017/07981.jpg')
img = cv2.Canny(img, 50, 100)
while True:
    cv2.imshow("frame", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        cv2.destroyAllWindows()
        break

In [235]:
for j in imgs[:1]:
        target_arr = np.load('../Datasets/hdd_data/target/%s' % (j + '.npy'), allow_pickle=True)
        sensor_arr = np.load('../Datasets/hdd_data/sensor/%s' % (j + '.npy'), allow_pickle=True)
        jpg_arr = ['../Datasets/hdd_data/camera/%s/%s' % (j, i) for i in os.listdir('hdd_data/camera/%s' % j)]
        sensor_acc = []
        count = 0
        for i in range(round(len(target_arr) * 0.8)):
            if sensor_arr[i][0] == 0 and sensor_arr[i-1][0] != 0:
                check = np.sum([sensor_arr[i+w][0] for w in range(1, 15)])
                if check != 0:
                    continue
                        
                count += 1
                #car stopped
                img = cv2.imread(jpg_arr[i])
                blob = cv2.dnn.blobFromImage(img,
                                     1 / 255, (320, 320), [0, 0, 0],
                                     1,
                                     crop=False)
                yolo.setInput(blob)
                layersNames = yolo.getLayerNames()
                outputNames = [(layersNames[i - 1])
                            for i in yolo.getUnconnectedOutLayers()]
                outputs = yolo.forward(outputNames)
                vehicles_index = [2, 3, 5, 7]
                vehicle_boxes = []
                height, width = img.shape[:2]
                for output in outputs:
                    for det in output:
                        scores = det[5:]
                        classId = np.argmax(scores)
                        confidence = scores[classId]
                        if classId in vehicles_index:
                            if confidence > 0.9:
                                w, h = int(det[2] * width), int(det[3] * height)
                                x, y = int((det[0] * width) -
                                        w / 2), int((det[1] * height) - h / 2)
                                if x < width * 0.7 and x > width * 0.3:
                                    vehicle_boxes.append([x, y, w, h])
                



print(count)

97


In [144]:
#for sensor_npy in os.listdir('hdd_data/sensor'):
sensors = os.listdir('../Datasets/hdd_data/sensor')
sensors.remove('README.md')
sensors.remove('.README.md.swp')
sensors.sort()
# for j in sensors:
#     arr = [i[0] for i in np.load('hdd_data/sensor/%s' % j)]
#     if len(arr):
#         print(np.max(arr))
#     else:
#         print('nan')

#print([i[7] for i in np.load('hdd_data/sensor/%s' % sensors[3])])
#print([i[0] for i in np.load('hdd_data/sensor/%s' % sensors[0])])

In [197]:
target = os.listdir('../Datasets/hdd_data/target')
target.remove('README.md')
target.remove('.README.md.swp')
target.sort()

for j in target:
    target_arr = np.load('../Datasets/hdd_data/target/%s' % j, allow_pickle=True)
    sensor_arr = np.load('../Datasets/hdd_data/sensor/%s' % j, allow_pickle=True)
    sensor_acc = []
    for i in range(round(len(target_arr) * 0.8)):
        #if target_arr[i] == 7:
        sensor_acc.append(sensor_arr[i][4])
    if len(sensor_acc):
        print(np.max(sensor_acc))
    else:
        print('nan')



# for j in target:
#     frame_count = 0
#     count = 0
#     arr = np.load('hdd_data/target/%s' % j, allow_pickle=True)
#     for i in range(round(len(arr) * 0.8)):
#         # if arr[i] == 2:
#         #     frame_count += 1
#         if arr[i] == 1 and arr[i-1] != 1:
#             count += 1
#     if count != 0:
#         #print(round(frame_count/count, 3))
#         print(count)
#     else:
#         print(0)

# print(target[3])
#np.load('hdd_data/target/%s' % target[0]).tolist().index(8)


6589.0
4768.04
6636.92
6229.6
5367.04
4288.84
5654.56
6852.56
6061.88
2707.48
4576.36
6301.48
3953.4
3977.36
3402.32
4193.0
3689.84
5534.76
3282.52
5103.48
4792.0
7331.76
0.0
2731.44
2995.0
5247.24
3617.96
4097.16
3162.72
2779.36
3282.52
2779.36
5558.72
3833.6
6996.32
6780.68
3809.64
5271.2
6253.56
6397.32
3713.8
3042.92
2563.72
3474.2
3426.28
1964.72
5582.68
5103.48
6636.92
5007.64
862.56
1797.0
6780.68
6708.8
5199.32
6972.36
6972.36
3138.76
2491.84
4384.68
6924.44
2611.64
6660.88
4240.92
3641.92
6732.76
3594.0
5630.6
5319.12
5846.24
5630.6
3210.64
7140.08
7068.2
5606.64
3833.6
5990.0
4983.68
6756.72
4384.68
6109.8
5582.68
6828.6
3426.28
2587.68
4935.76
4839.92
3258.56
5654.56
6541.08
6565.04
5534.76
6229.6
6133.76
5007.64
5798.32
6684.84
6061.88
4624.28
5271.2
4384.68
4216.96
6565.04
5295.16
6277.52
4911.8
4384.68
3234.6
6397.32
2803.32
6301.48
6780.68
2611.64
5462.88
3809.64
2923.12
5391.0
5367.04
5127.44
3090.84
3953.4
2755.4
6349.4
2300.16
2276.2
0.0
2348.08
1964.72
3258.56
6349.4

In [34]:
camera = os.listdir('../Datasets/hdd_data/camera')
camera.sort()
camera.remove('.DS_Store')
for i in camera:
    print(len(os.listdir("../Datasets/hdd_data/camera/%s" % i)))

9487
1326
10350
19474
15115
11790
12434
9923
10697
792
882
5011
4399
1735
2412
4031
3786
4277
3807
4365
4213
5237
2257
7853
9190
8871
5161
5603
5093
4726
4578
5060
3721
4089
14199
11334
13897
5783
9189
2824
4977
10058
3930
1890
3609
1011
1604
10153
28244
2443
1411
4649
6220
10046
10138
2943
3218
4709
1614
3350
4318
2206
15334
8526
4191
4312
1763
1642
4536
5740
6998
3637
11700
2447
2633
12680
6928
12230
15742
6250
22348
15637
2334
2331
2610
2135
1463
4518
5760
2250
1591
3721
10884
18045
4513
18324
14319
11615
12600
14414
7020
9675
7470
4500
9571
7251
14454
9122
19704
5922
9784
13230
10735
20115
15148
8980
24436
10823
13484
5992
13327
9300
22860
3765
18839
1105
18302
4760
14624
11085
9152
6515
20022
4304
11277
5836
30495
